In [ ]:
!pip install crewai crewai-tools transformers

  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3


In [ ]:
from crewai import Agent, Task, Crew, Process

In [ ]:
# Define the agents with roles, goals, and backstories to satisfy the model requirements.
research_agent = Agent(
    role="Research Assistant",
    goal="Extract the main points and important facts from the article.",
    backstory="You are a meticulous research assistant who carefully reads texts and extracts key points."
)
summarizer_agent = Agent(
    role="Summarizer",
    goal="Summarize the key points into a concise paragraph.",
    backstory="You are an expert summarizer who can condense information into a clear and brief summary."
)
writer_agent = Agent(
    role="Writer",
    goal="Organize the summary into a final report with a title and conclusion.",
    backstory="You are a creative writer with an eye for structure and clarity, ensuring the final output is polished."
)

# Example: Create tasks for each agent
research_task = Task(
    description="Read the article and identify the main points and important facts.",
    expected_output="A list of bullet points summarizing the key information from the article.",
    agent=research_agent
)
summarization_task = Task(
    description="Take the above bullet points and summarize them into a concise paragraph that captures the article's essence.",
    expected_output="A brief paragraph summarizing the article.",
    agent=summarizer_agent
)
writing_task = Task(
    description="Review the summary paragraph and format it with a clear title and a concluding sentence.",
    expected_output="A structured summary of the article with a title and conclusion.",
    agent=writer_agent
)

# Create a crew with a sequential process
crew = Crew(
    agents=[research_agent, summarizer_agent, writer_agent],
    tasks=[research_task, summarization_task, writing_task],
    process=Process.sequential,
    verbose=True
)

print("Agents defined successfully!")

Agents defined successfully!


In [ ]:
# Sample article text (as a multi-paragraph string)
article_text = """Artificial intelligence (AI) has made significant inroads in various sectors, transforming how we work and live.
One of the most notable impacts of AI is seen in healthcare, where machine learning algorithms assist doctors in diagnosing diseases faster and more accurately.

In the automotive industry, AI powers self-driving cars, analyzing traffic patterns in real-time to ensure passenger safety.
This technology also plays a crucial role in finance, with AI-driven algorithms detecting fraudulent transactions and enabling personalized banking services.

Education is another field being revolutionized by AI. Intelligent tutoring systems and personalized learning platforms adapt to individual student needs, making education more accessible and effective.

Despite these advancements, AI adoption also raises important questions.
Concerns about job displacement, ethical use of AI, and ensuring data privacy are at the forefront of public discourse as AI continues to evolve.

Overall, AI's growing influence across industries showcases its potential to tackle complex problems, but it also underscores the need for thoughtful integration to address the challenges that accompany these innovations.
"""
print("Article length (characters):", len(article_text))


Article length (characters): 1208


In [ ]:
import re

def extract_key_points(text):
    paragraphs = [p.strip() for p in text.split("\n\n") if p.strip()]
    key_points = []
    for para in paragraphs:
        sentences = re.split(r'(?<=\.)\s+', para.strip())
        if not sentences:
            continue
        main_sentence = max(sentences, key=len)
        point = main_sentence.strip()
        if point and point[-1] not in ".!?":
            point += "."
        key_points.append(point)
    return key_points

# Use the function on the article_text
key_points = extract_key_points(article_text)
print("Research Assistant Agent - Key Points:\n")
for i, point in enumerate(key_points, 1):
    print(f"{i}. {point}")

Research Assistant Agent - Key Points:

1. One of the most notable impacts of AI is seen in healthcare, where machine learning algorithms assist doctors in diagnosing diseases faster and more accurately.
2. This technology also plays a crucial role in finance, with AI-driven algorithms detecting fraudulent transactions and enabling personalized banking services.
3. Intelligent tutoring systems and personalized learning platforms adapt to individual student needs, making education more accessible and effective.
4. Concerns about job displacement, ethical use of AI, and ensuring data privacy are at the forefront of public discourse as AI continues to evolve.
5. Overall, AI's growing influence across industries showcases its potential to tackle complex problems, but it also underscores the need for thoughtful integration to address the challenges that accompany these innovations.


In [ ]:
from transformers import pipeline

# Initialize a summarization pipeline
summarizer_pipeline = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

# Prepare the input for summarization by joining the key points
input_text = " ".join(key_points)
summary_output = summarizer_pipeline(input_text, max_length=100, min_length=30, do_sample=False)
summary_text = summary_output[0]['summary_text']
print("Summarizer Agent - Summary:\n")
print(summary_text)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Summarizer Agent - Summary:

 AI's growing influence across industries showcases its potential to tackle complex problems, but it also underscores the need for thoughtful integration to address the challenges that accompany these innovations . Concerns about job displacement, ethical use of AI are at the forefront of public discourse as AI continues to evolve .


In [ ]:
# Writer agent formatting
title = "AI's Impact Across Industries: A Summary"
conclusion = "In conclusion, while AI offers tremendous benefits across sectors, addressing its challenges is crucial for its responsible adoption."

# Combine title, summary, and conclusion
final_report = f"# {title}\n\n{summary_text}\n\n{conclusion}"
print("Writer Agent - Final Output:\n")
print(final_report)

Writer Agent - Final Output:

# AI's Impact Across Industries: A Summary

 AI's growing influence across industries showcases its potential to tackle complex problems, but it also underscores the need for thoughtful integration to address the challenges that accompany these innovations . Concerns about job displacement, ethical use of AI are at the forefront of public discourse as AI continues to evolve .

In conclusion, while AI offers tremendous benefits across sectors, addressing its challenges is crucial for its responsible adoption.
